# Treinamento CIS - 1º Período (Regressão)


Nome: Gustavo M. Tonnera

Repositório: https://github.com/GMTonnera/ieee-cis-trainee

## Variáveis Globais e Imports

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [47]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
# Datasets
import io

TRAINING_DATAFRAME = pd.read_csv('/content/drive/MyDrive/datasets/linear-regression/train.csv')
TEST_DATAFRAME = pd.read_csv('/content/drive/MyDrive/datasets/linear-regression/test.csv')
pd.set_option("display.max_columns", None)

<ipython-input-48-27d96d7fb573>:4: DtypeWarning: Columns (0,21,24,29) have mixed types. Specify dtype option on import or set low_memory=False.
  TRAINING_DATAFRAME = pd.read_csv('/content/drive/MyDrive/datasets/linear-regression/train.csv')


## Funções Auxiliares

In [49]:
def checkNullValues(df):
    flag = False
    # Verificar se alguma coluna possui dados faltantes
    for column in df.columns.values:
        x = df[column].isnull().sum()
        if x > 0:
            print(f'{column} = {x}')
            flag = True

    if not flag:
        print("Nenhum dado faltante!")


def fillNullWithMode(df, columns):
    newDf = df.copy()
    print()
    for column in columns:
        mode_value = newDf[column].mode()[0]
        newDf[column] = newDf[column].fillna(mode_value)

    return newDf


def fillNullWithMedian(df, columns):
    newDf = df.copy()
    for column in columns:
        median_value = newDf[column].median()
        newDf[column] = newDf[column].fillna(median_value)

    return newDf

## Tarefas

### a) Fazer uma análise no dataset utilizando as ferramentas aprendidas no período.

In [50]:
TRAINING_DATAFRAME

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews_ltm,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,host_is_superhost,host_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,extra_people,minimum_nights,number_of_reviews,instant_bookable,amenities,property_type,room_type,cancellation_policy,price
0,947924,https://www.airbnb.com/rooms/947924,2.018082e+13,2018-08-16,LARGE LOVELY ROOM GREAT FOR GROUPS,NaN,This large lovely room is perfect for any grou...,This large lovely room is perfect for any grou...,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,5.0,-22.935381,-43.199022,6,1.0,1.0,4.0,$82.00,1,4.0,f,"{Internet,Wifi,Kitchen,""Free parking on premis...",House,Private room,strict_14_with_grace_period,$229.00
1,2747040,https://www.airbnb.com/rooms/2747040,2.019072e+13,2019-07-16,Alugo para a Copa do mundo,"Excelente apto, localizado a 500 metros de dis...",NaN,"Excelente apto, localizado a 500 metros de dis...",none,NaN,10.0,10.0,30.0,30.0,10.0,30.0,0.0,1.0,0.0,0.0,f,1.0,-22.912320,-43.219480,5,1.0,3.0,3.0,$0.00,10,0.0,f,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",Apartment,Entire home/apt,moderate,"$1,502.00"
2,13734316,https://www.airbnb.com/rooms/13734316,2.018112e+13,2018-11-15,Apartamento para temporada,Meu espaço é perto de Vila olímpica dos atleta...,NaN,Meu espaço é perto de Vila olímpica dos atleta...,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1.0,-22.965607,-43.409884,4,1.0,2.0,3.0,$0.00,30,0.0,t,"{TV,""Air conditioning"",Pool,Kitchen,""Free park...",Apartment,Entire home/apt,flexible,$569.00
3,30979175,https://www.airbnb.com/rooms/30979175,2.019092e+13,2019-09-24,2-Ensuite flat with stunning sea view!,Enjoy the most beautiful sight of Recreio dos ...,The flat is newly refurbished and has a modern...,Enjoy the most beautiful sight of Recreio dos ...,none,NaN,2.0,4.0,15.0,15.0,2.4,15.0,20.0,1.0,0.0,0.0,t,1.0,-23.030740,-43.477210,5,2.5,2.0,4.0,$75.00,2,20.0,f,"{TV,""Cable TV"",Wifi,""Air conditioning"",Pool,Ki...",Apartment,Entire home/apt,moderate,$499.00
4,3742926,https://www.airbnb.com/rooms/3742926,2.018121e+13,2018-12-14,Condomino,"Rio-Stadtteil JACAREPAGUA-CURICICA, kein Touri...",NaN,"Rio-Stadtteil JACAREPAGUA-CURICICA, kein Touri...",none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,2.0,-22.949252,-43.388894,4,2.0,2.0,3.0,$88.00,7,0.0,f,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",House,Private room,flexible,$132.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588086,17574111,https://www.airbnb.com/rooms/17574111,2.019062e+13,2019-06-20,Sala e Quarto Mais Charmoso de Copacabana,"À 2 quadras da praia Copacabana; Linda Vista, ...",O conforto e a localização são o ponto alto de...,"À 2 quadras da praia Copacabana; Linda Vista, ...",none,Copacabana é o bairro mais típico e boêmio do ...,5.0,5.0,180.0,180.0,5.0,180.0,19.0,4.0,0.0,0.0,t,4.0,-22.980850,-43.194040,4,1.0,1.0,3.0,$0.00,5,36.0,t,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",Apartment,Entire home/apt,strict_14_with_grace_period,$100.00
588087,2763759,https://www.airbnb.com/rooms/2763759,2.018082e+13,2018-08-16,artistico apartamento ipanema,"Apartamento muito bem decorado e projetado, 2 ...","Apartamento cuidadosamente decorado, charmoso ...","Apartamento muito bem decorado e projetado, 2 ...",none,tudo o que voce precisa voce encontra a poucos...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1.0,-22.982440,-43.200213,4,3.0,2.0,2.0,$400.00,3,0.0,t,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",Apartment,Entire home/apt,strict_14_with_grace_period,"$1,200.00"
588088,11732634,https://www.ai

#### Labels do dataset:

- id: Identificador único da listagem.

- listing_url: URL da página da listagem na plataforma.

- scrape_id: Identificador único da operação de coleta de dados (scrape).

- last_scraped: Data da última vez que os dados foram coletados.

- name: Nome da propriedade.

- summary: Resumo curto da propriedade.

- space: Descrição do espaço da propriedade, incluindo áreas comuns e privadas.

- description: Descrição detalhada da propriedade, incluindo comodidades e características.

- experiences_offered: Indica se experiências (atividades) são oferecidas juntamente com a estadia.

- neighborhood_overview: Visão geral do bairro onde a propriedade está localizada.

- minimum_minimum_nights: Número mínimo de noites exigido pela reserva mínima mais baixa permitida.

- maximum_minimum_nights: Número máximo de noites exigido pela reserva mínima mais baixa permitida.

- minimum_maximum_nights: Número mínimo de noites exigido pela reserva máxima mais alta permitida.

- maximum_maximum_nights: Número máximo de noites exigido pela reserva máxima mais alta permitida.

- minimum_nights_avg_ntm: Média do número mínimo de noites exigido para reservas no próximo mês.

- maximum_nights_avg_ntm: Média do número máximo de noites permitido para reservas no próximo mês.

- number_of_reviews_ltm: Número de avaliações recebidas nos últimos doze meses.

- calculated_host_listings_count_entire_homes: Número de propriedades inteiras listadas pelo anfitrião.

- calculated_host_listings_count_private_rooms: Número de quartos privados listados pelo anfitrião.

- calculated_host_listings_count_shared_rooms: Número de quartos compartilhados listados pelo anfitrião.

- host_is_superhost: Indica se o anfitrião é um "superhost" ou não. Um superhost é um anfitrião experiente e altamente avaliado que proporciona estadias excepcionais para seus hóspedes.

- host_listings_count: Número total de listagens (propriedades) que o anfitrião possui na plataforma.

- latitude: Latitude da localização da propriedade.

- longitude: Longitude da localização da propriedade.

- accommodates: Número de hóspedes que a propriedade pode acomodar.

- bathrooms: Número de banheiros na propriedade.

- bedrooms: Número de quartos na propriedade.

- beds: Número de camas na propriedade.

- extra_people: Custo adicional por hóspede extra acima do número de hóspedes permitido.

- minimum_nights: Número mínimo de noites que um hóspede deve reservar.

- number_of_reviews: Número total de avaliações recebidas pela propriedade.

- instant_bookable: Indica se a propriedade pode ser reservada instantaneamente, sem a necessidade de aprovação prévia do anfitrião.

- amenities: Amenidades disponíveis na propriedade.

- property_type - Indica o tipo de propriedade.

- room_type - Indica o tipo de quarto.

- cancellation_policy: Indica o tipo de política de cancelamento.

As labels "listing_url", "scrape_id", "last_scraped", "name", "summary", "space", "description", "experiences_offered" e "neighborhood_overview" são labels com valores no formato de string que descrevem informações do local. Essas labels não serão usadas no modelo de regressão linear, já que são dados do tipo string não categóricas.

A label "amenities" é um array de strings que descreve aminidades que a propriedade possui, como televisão, internet, etc. Essa label até poderia ser codificado em um dado numérico usando One Hot Coding, porém a dimensão da entrada do modelo seria gigantesca: cada amenidade aumentaria o tamanha da dimensão da entrada em 1. Por esse problema, essa variável também não será usada no modelo.

As labels "host_is_superhost" e "instant_bookable" são labels booleanas e são classificadas como variáveis categóricas nominais. Assim, podem ser codificadas, por meio do One Hot Encoding, a fim de serem utilizadas no modelo de regressão linear.

As labels "property_type", "room_type" e "cancellation_policy" são variáveis categóricas nominais no formato de string e também podem ser codificada pelo One Hot Encoding.

As demais labels são variáveis numéricas contínuas ou discretas.

#### Variáveis do dataset

- price: Preço da propriedade por noite (target).

### b) Demonstrar como pré-processar e representar os diferentes tipos de dados.

#### Remover colunas não significantes

Primeiramente, precisamos retirar as colunas que não conseguimos codificar para ser utilizadas no modelo. Essas variáveis são: "id", "listing_url", "scrape_id", "last_scraped", "name", "summary", "space", "description", "experiences_offered", "neighborhood_overview" e "amenities".

In [51]:
TRAINING_DATAFRAME = TRAINING_DATAFRAME.drop(
    columns=[
        "id",
        "listing_url",
        "scrape_id",
        "last_scraped",
        "name",
        "summary",
        "space",
        "description",
        "experiences_offered",
        "neighborhood_overview",
        "amenities"
    ]
)

TRAINING_DATAFRAME

,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews_ltm,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,host_is_superhost,host_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,extra_people,minimum_nights,number_of_reviews,instant_bookable,property_type,room_type,cancellation_policy,price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,5.0,-22.935381,-43.199022,6,1.0,1.0,4.0,$82.00,1,4.0,f,House,Private room,strict_14_with_grace_period,$229.00
1,10.0,10.0,30.0,30.0,10.0,30.0,0.0,1.0,0.0,0.0,f,1.0,-22.912320,-43.219480,5,1.0,3.0,3.0,$0.00,10,0.0,f,Apartment,Entire home/apt,moderate,"$1,502.00"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1.0,-22.965607,-43.409884,4,1.0,2.0,3.0,$0.00,30,0.0,t,Apartment,Entire home/apt,flexible,$569.00
3,2.0,4.0,15.0,15.0,2.4,15.0,20.0,1.0,0.0,0.0,t,1.0,-23.030740,-43.477210,5,2.5,2.0,4.0,$75.00,2,20.0,f,Apartment,Entire home/apt,moderate,$499.00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,2.0,-22.949252,-43.388894,4,2.0,2.0,3.0,$88.00,7,0.0,f,House,Private room,flexible,$132.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588086,5.0,5.0,180.0,180.0,5.0,180.0,19.0,4.0,0.0,0.0,t,4.0,-22.980850,-43.194040,4,1.0,1.0,3.0,$0.00,5,36.0,t,Apartment,Entire home/apt,strict_14_with_grace_period,$100.00
588087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1.0,-22.982440,-43.200213,4,3.0,2.0,2.0,$400.00,3,0.0,t,Apartment,Entire home/apt,strict_14_with_grace_period,"$1,200.00"
588088,1.0,1.0,1125.0,1125.0,1.0,1125.0,0.0,1.0,0.0,0.0,f,2.0,-22.913180,-43.378300,4,2.0,2.0,2.0,$0.00,1,0.0,f,Apartment,Entire home/apt,flexible,$298.00
588089,5.0,5.0,30.0,30.0,5.0,30.0,0.0,1.0,0.0,0.0,f,1.0,-22.990680,-43.299580,10,7.0,6.0,10.0,$0.00,5,0.0,f,House,Entire home/apt,flexible,"$7,901.00"


#### Convertendo tipos de dados das colunas

Antes de verificar se existem dados faltantes, precismos verificar se os tipos de dados da colunas são os tipos que precismos.

In [52]:
TRAINING_DATAFRAME.dtypes

minimum_minimum_nights                          float64
maximum_minimum_nights                          float64
minimum_maximum_nights                          float64
maximum_maximum_nights                          float64
minimum_nights_avg_ntm                          float64
maximum_nights_avg_ntm                          float64
number_of_reviews_ltm                           float64
calculated_host_listings_count_entire_homes     float64
calculated_host_listings_count_private_rooms    float64
calculated_host_listings_count_shared_rooms     float64
host_is_superhost                                object
host_listings_count                              object
latitude                                        float64
longitude                                       float64
accommodates                                     object
bathrooms                                       float64
bedrooms                                        float64
beds                                            

Podemos observar que algumas colunas com labels numéricas estão sendo representadas por strings, mas precisamos que esses dados sejam representados por números. Então, faremos essa conversão a seguir. Durante as tentativas de converter os tipos das colunas, me deparei com uma linha especifíca do dataset com dados completamente sem sentido: a linha com index 213883. Devido a isso, essa linha foi removida do dataset.

In [53]:
print(TRAINING_DATAFRAME.iloc[213883])
TRAINING_DATAFRAME = TRAINING_DATAFRAME.drop(213883)
TRAINING_DATAFRAME.iloc[213883]
TRAINING_DATAFRAME = TRAINING_DATAFRAME.reset_index(drop=True)

minimum_minimum_nights                                 NaN
maximum_minimum_nights                                 NaN
minimum_maximum_nights                                 NaN
maximum_maximum_nights                                 NaN
minimum_nights_avg_ntm                                 NaN
maximum_nights_avg_ntm                                 NaN
number_of_reviews_ltm                                  NaN
calculated_host_listings_count_entire_homes            NaN
calculated_host_listings_count_private_rooms           NaN
calculated_host_listings_count_shared_rooms            NaN
host_is_superhost                                        3
host_listings_count                               Real Bed
latitude                                              43.0
longitude                                             43.0
accommodates                                    2019-03-06
bathrooms                                            100.0
bedrooms                                              10

Para conseguir converter os valores das colunas "extra_people" e "price" para float, precisamos remover o símbolo "$" dos valores. Além disso, alguns valores da coluna "extra_people" estão formatados incorretamente no formato "1,100.00". Então, precisamos remover a vírgula desses valores para poder convertê-los para valores numéricos.

In [54]:
TRAINING_DATAFRAME['extra_people'] = TRAINING_DATAFRAME['extra_people'].map(lambda x: x.lstrip('$').replace(',', ''))
TRAINING_DATAFRAME['price'] = TRAINING_DATAFRAME['price'].map(lambda x: x.lstrip('$').replace(',', ''))

Agora, finalmente, podemos realizar as conversões de tipos.

In [55]:
convert_dict = {
    "host_listings_count": float,
    "accommodates": int,
    "extra_people": float,
    "minimum_nights": int,
    "price": float
}

TRAINING_DATAFRAME = TRAINING_DATAFRAME.astype(convert_dict)

TRAINING_DATAFRAME.dtypes

minimum_minimum_nights                          float64
maximum_minimum_nights                          float64
minimum_maximum_nights                          float64
maximum_maximum_nights                          float64
minimum_nights_avg_ntm                          float64
maximum_nights_avg_ntm                          float64
number_of_reviews_ltm                           float64
calculated_host_listings_count_entire_homes     float64
calculated_host_listings_count_private_rooms    float64
calculated_host_listings_count_shared_rooms     float64
host_is_superhost                                object
host_listings_count                             float64
latitude                                        float64
longitude                                       float64
accommodates                                      int64
bathrooms                                       float64
bedrooms                                        float64
beds                                            

#### Tratar dados faltantes

Agora, é necessário verificar se existem valores nulos nas colunas restantes.

In [56]:
checkNullValues(TRAINING_DATAFRAME)

minimum_minimum_nights = 136969
maximum_minimum_nights = 136969
minimum_maximum_nights = 136969
maximum_maximum_nights = 136969
minimum_nights_avg_ntm = 136969
maximum_nights_avg_ntm = 136969
number_of_reviews_ltm = 136969
calculated_host_listings_count_entire_homes = 136969
calculated_host_listings_count_private_rooms = 136969
calculated_host_listings_count_shared_rooms = 136969
host_is_superhost = 294
host_listings_count = 294
bathrooms = 1126
bedrooms = 592
beds = 1759


Observa-se que existem várias colunas com dados faltantes. Par tratá-los, usaremos a seguinte estratégia:

- Para labels numéricas, substituiremos os dados faltantes pela mediana dos valores da coluna, já que a mediana não sensível a outliers;

- Para labels categóricas nominais, substituiremos os dados faltantes pela categoria mais representativa, já que essas labelsnão possuem muitos dados faltantes. Assim, a proporção entre as categorias não deve sofrer grandes mudanças.  

##### Tratando as variáveis numéricas:

In [57]:
numericalLabels = [
    "minimum_minimum_nights",
    "maximum_minimum_nights",
    "minimum_maximum_nights",
    "maximum_maximum_nights",
    "minimum_nights_avg_ntm",
    "maximum_nights_avg_ntm",
    "number_of_reviews_ltm",
    "calculated_host_listings_count_entire_homes",
    "calculated_host_listings_count_private_rooms",
    "calculated_host_listings_count_shared_rooms",
    "host_listings_count",
    "bathrooms",
    "bedrooms",
    "beds"
]

TRAINING_DATAFRAME = fillNullWithMedian(TRAINING_DATAFRAME, numericalLabels)

##### Tratando as variáveis categóricas nominais:

In [58]:
categoricalLabels = [
    "host_is_superhost"
]

TRAINING_DATAFRAME = fillNullWithMode(TRAINING_DATAFRAME, categoricalLabels)

##### Verificar se ainda restam dados faltantes

In [59]:
checkNullValues(TRAINING_DATAFRAME)

Nenhum dado faltante!


#### Aplicar escalas nas labels numéricas

Afim de aplicar uma escala nos dados e centralizar os dados, aplicaremos o procedimento de "standardization" nas labels numéricas.

In [60]:
from sklearn import preprocessing
labels = [
    "minimum_minimum_nights",
    "maximum_minimum_nights",
    "minimum_maximum_nights",
    "maximum_maximum_nights",
    "minimum_nights_avg_ntm",
    "maximum_nights_avg_ntm",
    "number_of_reviews_ltm",
    "calculated_host_listings_count_entire_homes",
    "calculated_host_listings_count_private_rooms",
    "calculated_host_listings_count_shared_rooms",
    "host_listings_count",
    "latitude","longitude",
    "accommodates",
    "bathrooms",
    "bedrooms",
    "beds",
    "extra_people",
    "minimum_nights",
    "number_of_reviews"
]

newDf = TRAINING_DATAFRAME.copy()
scaler = preprocessing.StandardScaler().fit(newDf[labels].to_numpy())
newDf2 = scaler.transform(newDf[labels].to_numpy())
newDf2 = pd.DataFrame(newDf2, columns=labels)
for column in labels:
    newDf[column] = newDf2[column]

TRAINING_DATAFRAME = newDf
TRAINING_DATAFRAME

,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews_ltm,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,host_is_superhost,host_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,extra_people,minimum_nights,number_of_reviews,instant_bookable,property_type,room_type,cancellation_policy,price
0,-0.108236,-0.084356,-0.002958,-0.002959,-0.104302,-0.002959,-0.347359,-0.168537,-0.357091,-0.106264,f,-0.076085,0.855728,0.562993,0.683209,-0.581643,-0.599910,0.683443,0.419266,-0.166158,-0.178510,f,House,Private room,strict_14_with_grace_period,229.0
1,0.321028,0.278965,-0.003333,-0.003335,0.311393,-0.003334,-0.347359,-0.168537,-0.357091,-0.106264,f,-0.157871,1.516306,0.352947,0.303055,-0.581643,1.263494,0.198146,-0.404597,0.236245,-0.357222,f,Apartment,Entire home/apt,moderate,1502.0
2,-0.108236,-0.084356,-0.002958,-0.002959,-0.104302,-0.002959,-0.347359,-0.168537,-0.357091,-0.106264,f,-0.157871,-0.010106,-1.601970,-0.077098,-0.581643,0.331792,0.198146,-0.404597,1.130474,-0.357222,t,Apartment,Entire home/apt,flexible,569.0
3,-0.108236,-0.032453,-0.003339,-0.003340,-0.093643,-0.003339,2.654521,-0.168537,-0.357091,-0.106264,t,-0.157871,-1.875878,-2.293217,0.303055,0.680706,0.331792,0.683443,0.348937,-0.121447,0.536340,f,Apartment,Entire home/apt,moderate,499.0
4,-0.108236,-0.084356,-0.002958,-0.002959,-0.104302,-0.002959,-0.347359,-0.168537,-0.357091,-0.106264,f,-0.137425,0.458388,-1.386457,-0.077098,0.259923,0.331792,0.198146,0.479549,0.102110,-0.357222,f,House,Private room,flexible,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588085,0.052738,0.019450,-0.003282,-0.003283,0.044922,-0.003283,2.504427,-0.039416,-0.357091,-0.106264,t,-0.096532,-0.446761,0.614144,-0.077098,-0.581643,-0.599910,0.198146,-0.404597,0.012688,1.251190,t,Apartment,Entire home/apt,strict_14_with_grace_period,100.0
588086,-0.108236,-0.084356,-0.002958,-0.002959,-0.104302,-0.002959,-0.347359,-0.168537,-0.357091,-0.106264,f,-0.157871,-0.492308,0.550762,-0.077098,1.101490,0.331792,-0.287151,3.614248,-0.076735,-0.357222,t,Apartment,Entire home/apt,strict_14_with_grace_period,1200.0
588087,-0.161895,-0.188161,-0.002958,-0.002959,-0.168255,-0.002959,-0.347359,-0.168537,-0.357091,-0.106264,f,-0.137425,1.491671,-1.277689,-0.077098,0.259923,0.331792,-0.287151,-0.404597,-0.166158,-0.357222,f,Apartment,Entire home/apt,flexible,298.0
588088,0.052738,0.019450,-0.003333,-0.003335,0.044922,-0.003334,-0.347359,-0.168537,-0.357091,-0.106264,f,-0.157871,-0.728345,-0.469455,2.203822,4.467755,4.058601,3.595228,-0.404597,0.012688,-0.357222,f,House,Entire home/apt,flexible,7901.0


### c) Demonstrar a utilização de técnicas como one-hot encoding, redução de dimensionalidade e PCA;

#### One-Hoting Encoding

O one-hot enconding é uma técnica usada para transformar varáveis categóricas nominais em variáveis numéricas. Aplicaremos essa transformação nas seguintes labels: "property_type", "room_type", "cancellation_policy", "host_is_superhost" e "instant_bookable".

Como as labels "host_is_superhost" e "instant_bookable" possuem apenas duas categorias booleanas (true e false), basta converter os valores true para 1 e false para 0.

As labels "room_type" e "cancellation_policy" possuem poucas categorias, então podemos aplicar o one-hot enconding sem problemas.

A label "property_type" possui 47 categorias, o que resultaria em um aumeno de 47 na dimensionalidade na entrada do modelo aplicando o one-hot encoding. Porém, em seguida, aplicaremos o PCA para diminuir a dimensionalidade dos dados de entrada. Então, também aplicaremos o one-hot encoding nessa label.

In [61]:
# host_is_superhost
TRAINING_DATAFRAME["host_is_superhost"] = TRAINING_DATAFRAME["host_is_superhost"].replace('t', 1)
TRAINING_DATAFRAME["host_is_superhost"] = TRAINING_DATAFRAME["host_is_superhost"].replace('f', 0)

# instant_bookable
TRAINING_DATAFRAME["instant_bookable"] = TRAINING_DATAFRAME["instant_bookable"].replace('t', 1)
TRAINING_DATAFRAME["instant_bookable"] = TRAINING_DATAFRAME["instant_bookable"].replace('f', 0)

# room_type
one_hot = pd.get_dummies(TRAINING_DATAFRAME['room_type'])
TRAINING_DATAFRAME = TRAINING_DATAFRAME.drop("room_type", axis=1)
TRAINING_DATAFRAME = TRAINING_DATAFRAME.join(one_hot)

# cancellation_policy
one_hot = pd.get_dummies(TRAINING_DATAFRAME['cancellation_policy'])
TRAINING_DATAFRAME = TRAINING_DATAFRAME.drop("cancellation_policy", axis=1)
TRAINING_DATAFRAME = TRAINING_DATAFRAME.join(one_hot)

# property_type
one_hot = pd.get_dummies(TRAINING_DATAFRAME['property_type'])
TRAINING_DATAFRAME = TRAINING_DATAFRAME.drop("property_type", axis=1)
TRAINING_DATAFRAME = TRAINING_DATAFRAME.join(one_hot)

# Mover a coluna price para o final
price = TRAINING_DATAFRAME.pop("price")
TRAINING_DATAFRAME.insert(len(TRAINING_DATAFRAME.columns),"price", price)

As colunas foram codificadas como tipo booleano, então precisamos converter os valores False para 0 e os valores True para 1.

In [62]:
for column in TRAINING_DATAFRAME.columns[22:]:
    TRAINING_DATAFRAME[column] = TRAINING_DATAFRAME[column].replace(True, 1)
    TRAINING_DATAFRAME[column] = TRAINING_DATAFRAME[column].replace(False, 0)

TRAINING_DATAFRAME

,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews_ltm,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,host_is_superhost,host_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,extra_people,minimum_nights,number_of_reviews,instant_bookable,Entire home/apt,Hotel room,Private room,Shared room,flexible,moderate,strict,strict_14_with_grace_period,super_strict_30,super_strict_60,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Cabin,Camper/RV,Campsite,Casa particular,Casa particular (Cuba),Castle,Chalet,Condominium,Cottage,Dome house,Dorm,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Houseboat,Hut,Igloo,Island,Lighthouse,Loft,Nature lodge,Other,Parking Space,Pension (South Korea),Plane,Resort,Serviced apartment,Tent,Tiny house,Tipi,Townhouse,Train,Treehouse,Vacation home,Villa,Yurt,price
0,-0.108236,-0.084356,-0.002958,-0.002959,-0.104302,-0.002959,-0.347359,-0.168537,-0.357091,-0.106264,0,-0.076085,0.855728,0.562993,0.683209,-0.581643,-0.599910,0.683443,0.419266,-0.166158,-0.178510,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,229.0
1,0.321028,0.278965,-0.003333,-0.003335,0.311393,-0.003334,-0.347359,-0.168537,-0.357091,-0.106264,0,-0.157871,1.516306,0.352947,0.303055,-0.581643,1.263494,0.198146,-0.404597,0.236245,-0.357222,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1502.0
2,-0.108236,-0.084356,-0.002958,-0.002959,-0.104302,-0.002959,-0.347359,-0.168537,-0.357091,-0.106264,0,-0.157871,-0.010106,-1.601970,-0.077098,-0.581643,0.331792,0.198146,-0.404597,1.130474,-0.357222,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,569.0
3,-0.108236,-0.032453,-0.003339,-0.003340,-0.093643,-0.003339,2.654521,-0.168537,-0.357091,-0.106264,1,-0.157871,-1.875878,-2.293217,0.303055,0.680706,0.331792,0.683443,0.348937,-0.121447,0.536340,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,499.0
4,-0.108236,-0.084356,-0.002958,-0.002959,-0.104302,-0.002959,-0.347359,-0.168537,-0.357091,-0.106264,0,-0.137425,0.458388,-1.386457,-0.077098,0.259923,0.331792,0.198146,0.479549,0.102110,-0.357222,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588085,0.052738,0.019450,-0.003282,-0.003283,0.044922,-0.003283,2.504427,-0.039416,-0.357091,-0.106264,1,-0.096532,-0.446761,0.614144,-0.077098,-0.581643,-0.599910,0.198146,-0.404597,0.012688,1.251190,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0
588086,-0.108236,-0.084356,-0.002958,-0.002959,-0.104302,-0.002959,-0.347359,-0.168537,-0.357091,-0.106264,0,-0.157871,-0.492308,0.550762,-0.077098,1.101490,0.331792,-0.287151,3.614248,-0.076735,-0.357222,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1200.0
588087,-0.161895,-0.188161,-0.002958,-0.002959,-0.168255,-0.002959,-0.347359,-0.168537,-0.357091,-0.106264,0,-0.137425,1.491671,-1.277689,-0.077098,0.259923,0.331792,-0.287151,-0.404597,-0.166158,-0.357222,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,298.0
588088,0.052738,0.019450,-0.003333,-0.003335,0.04

#### PCA

Agora que todas as labels são variáveis numéricas, podemos aplicar o PCA para verificar quais delas mais influenciam na variação do preço.

In [63]:
N_COMPONENTS = 20
y = TRAINING_DATAFRAME.pop("price")
x_training = TRAINING_DATAFRAME.to_numpy()
pca1 = PCA(n_components=N_COMPONENTS)
principalComponents1 = pca1.fit_transform(x_training)

In [64]:
new_training_data = pd.DataFrame(
    data = principalComponents1,
    columns = [f"PC{i}" for i in range(N_COMPONENTS)])

new_training_data

,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19
0,-0.262681,0.112701,-0.031170,-0.194771,-0.218798,1.151223,0.346954,0.363819,0.114955,0.239208,-0.863186,-0.462250,-1.032025,0.536842,0.191813,-0.054692,-0.937122,0.358411,-0.982215,-0.260123
1,0.634188,-0.243631,0.041502,-0.304357,-0.358197,1.581813,-0.318055,-0.488398,0.114846,0.446200,-0.541493,-0.466750,0.521431,-1.061497,-0.247293,-0.073597,0.052093,0.239690,0.101021,1.023601
2,0.445674,-0.219748,0.030959,-0.615512,-0.686580,-0.906188,-0.488620,-0.471115,-0.360166,1.011952,-0.718973,0.194293,0.286376,-0.395937,0.028480,1.023903,0.527412,-0.573101,0.222128,-0.092967
3,0.043625,-1.459552,0.298020,-0.420746,2.262474,-2.715437,-0.174395,0.157446,-0.313443,0.711485,-0.008646,0.322762,0.003255,0.174191,-1.461202,0.035680,0.209958,0.874491,0.186987,0.814407
4,-0.004129,-0.458281,0.073455,-0.706874,-0.832314,-0.496606,0.227657,0.162801,0.321654,1.618970,0.155114,0.221378,-0.490299,-0.206536,0.239504,0.142573,-0.474842,0.438620,-0.752803,-0.198421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588085,-0.144344,0.728813,-0.121428,0.388521,2.878601,-0.167135,-0.297918,-0.347287,-0.448051,-0.359653,-0.412408,0.066394,-0.090946,0.345134,-0.969267,0.065848,-0.068810,-0.360759,0.079979,-0.369555
588086,0.032792,-0.847940,0.162162,-0.488630,0.062135,0.330735,0.545291,3.641619,0.205859,-0.318020,0.968673,-0.090745,0.168618,0.233344,-0.198195,-0.018141,0.072609,-0.810431,0.238930,-0.240841
588087,-0.286112,-0.194308,0.022588,-0.599571,-0.826567,0.289538,-0.345835,-0.615683,0.072050,1.643268,0.172778,-0.374770,0.484154,-0.234996,0.053179,-0.026155,0.620172,0.428617,0.462769,-0.152033
588088,1.299579,-6.757794,1.323464,-0.810121,0.090634,0.619006,0.277436,-1.257248,0.977526,0.084018,1.582562,0.851078,-0.798864,-0.483463,0.009823,-0.067446,-0.042977,0.079217,-0.240034,-0.092525


In [65]:
print('Explained variability per principal component: {}'.format(pca1.explained_variance_ratio_))
print(f"Sum of variation: {sum(pca1.explained_variance_ratio_)}" )

Explained variability per principal component: [0.17198945 0.13946671 0.13761251 0.08547095 0.08258361 0.05838219
 0.05160994 0.04542346 0.04092639 0.02938935 0.02492157 0.01643613
 0.01564609 0.01361622 0.012629   0.01102052 0.01077759 0.01022113
 0.01005779 0.00941795]
Sum of variation: 0.9775985417861035


#### i) Como a redução da dimensionalidade afetou no desempenho do modelo?

A redução de dimensionalidade faz com que o modelo se torne menos complexo e facilite sua convergência.

### d) Aplicar regressão nos dados - dividir o dataset em treinamento e validação e aplicar, pelo menos, 3 algoritmos distintos de regressão para prever o preço do imóvel. Testar e visualizar os resultados, comparando o desempenho dos modelos treinados.

#### Separar o dataset em treinamento e validação

In [66]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(new_training_data, y, test_size=0.2, random_state=57)

O dataset já foi separado em treinamento e validação (arquivos "train.data" e "test.data"). Os algoritmos escolhidos de regressão foram:

- Linear Regression;

- Logistic Regression;

- Bayesian Ridge Regression.

#### Linear Regression

In [67]:
from sklearn.linear_model import LinearRegression

reg1 = LinearRegression()
reg1.fit(x_train, y_train)
reg1.score(x_test, y_test)

0.13482980630107666

#### Logistic Regression

In [68]:
# from sklearn.linear_model import LogisticRegression

# reg2 = LogisticRegression()
# reg2.fit(x_train[0:100000], y_train[0:100000])
# reg2.score(x_test, y_test)

#### Bayesian Ridge Regression

In [69]:
from sklearn import linear_model

reg3 = linear_model.BayesianRidge()
reg3.fit(x_train, y_train)
reg3.score(x_test, y_test)

0.13483975501586476

#### Resultados

Percebe-se que o modelo de regressivo Bayesian Ridge Regression performou melhor, mas apresenta uma performance bem ruim. Além disso, tentei de tudo para fazer o modelo de Rgressão Logística convergir, mas ele simplesmente não converge.

### e) Avaliar o desempenho do(s) modelo(s) treinado(s). A métrica da competição será o erro quadrático médio (MSE), porém, recomenda-se a avaliação do modelo por outras métricas adicionais, como o R² e o erro médio absoluto (MAE).

#### Linear Regression

In [74]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

prediction = reg1.predict(x_test)
print(f'MSE = {mean_squared_error(y_test, prediction)}')
print(f'R^2 = {r2_score(y_test, prediction)}')
print(f'MAE = {mean_absolute_error(y_test, prediction)}')

MSE = 2468729.084655033
R^2 = 0.13482980630107666
MAE = 500.9589484165189


#### Bayesian Ridge Regression

In [75]:
prediction = reg3.predict(x_test)
print(f'MSE = {mean_squared_error(y_test, prediction)}')
print(f'R^2 = {r2_score(y_test, prediction)}')
print(f'MAE = {mean_absolute_error(y_test, prediction)}')

MSE = 2468700.6963890814
R^2 = 0.13483975501586476
MAE = 500.9207138697439
